This notebook analyses the coverage across genes of the sequenced strains. Coverages are normalised by gene and a duplication-calling cutoff is set as 1.5 times the mode of coverage across strains in the given gene.

To get more contiguous duplications a segmentation procedure was applied to the called duplications. For this purpose the Viterbi algorithm for finding the most likely sequence of states in a Hidden Markov Model was used.

In [90]:
import pandas as pd
import numpy as np
from Bio import SeqIO
from concurrent.futures import ProcessPoolExecutor

In [43]:
genome = SeqIO.read("../Data/Mutation_data/MG1655.gb", "genbank")

position_dict = {}
for feat in genome.features:
    if "locus_tag" in feat.qualifiers:
        position_dict[feat.qualifiers["locus_tag"][0]] = feat.location.start.position

In [81]:
coverage = pd.read_csv("../Data/Mutation_data/coverage_FC_by_gene.tab.tsv", sep=" ", index_col=0)
names = coverage.columns
fixed_names = []
for name in names:
    ale = name.split("_")[0]
    name = name.split("_")[-3]
    if name.startswith("ALE-"):
        name = name[4:]
    name = name.replace("HDMA", "HMDA")
    name = name.replace("Glut-", "GLUT")
    name = name.replace("PUTR-", "PUTR")
    fixed_names.append(ale + "_" + name)
#coverage.columns = fixed_names
coverage.to_csv("../Data/Mutation_data/cleaned_coverage_by_gene.tsv", sep="\t")

In [82]:
# Normalise coverage by gene (divide by gene median)
norm_coverage = (coverage.T / coverage.T.median(0)).T

In [83]:
def find_mode(nums, binsize=0.005):
    """Find the mode of a distribution."""
    min_val = nums.min()
    max_val = nums.max()
    bins = np.array([0 for i in range(int((max_val - min_val) // binsize) + 1)])
    for num in nums:
        bin_num = int((num - min_val) // binsize)
        bins[bin_num] += 1
    return bins.argmax() * binsize + min_val

def cutoff_coverages(ser):
    """Convert a series to binary, denoting whether each point is greater or less than the mode of the series"""
    peak = find_mode(ser)
    return (ser > peak * 1.5).astype("int")

def segment_series(ser):
    """Segment a binary series using the Viterbi algorithm to find the most likely trajectory."""
    if ser.name not in duplications:
        print(ser.name)
        return ser
    peak = find_mode(ser.astype("int"))
    obs = np.array([stats.norm(peak, 0.2), stats.norm(2*peak, 0.2)])
    decoder = Decoder(start_probability, trans_mat, obs)
    return decoder.Decode(ser)

In [84]:
# Cutoff normalised coverage
duplications = norm_coverage.apply(cutoff_coverages)


dup_rate = duplications.sum().sum() / (duplications.shape[0] * duplications.shape[1])
transition_rate = 0.001 / 11

trans_mat = np.array([[1-transition_rate, transition_rate], [transition_rate, 1-transition_rate]])
start_probability = np.array([[1 - dup_rate, dup_rate]]).T

false_positive = 0.01
false_negative = 0.5
obs_mat = np.array([[1-false_positive, false_positive],
                    [false_negative, 1-false_negative]])
obs_mat = obs_mat * 1.2 # Scale to avoid numerical underflow

In [85]:
# Add the genome positions of each gene
duplications["pos"] = duplications.index.map(position_dict.get) 

# Calculate "length" of each gene (distance to next gene, measured in 100 bp's)
duplications["length"] = (
    pd.Series(list(duplications["pos"][1:]) + [len(genome)], index=duplications.index) - duplications["pos"]
).fillna(0) // 100 + 1
duplications["length"] = duplications["length"].astype("int")

In [86]:
# Expand genes according to their length
data = []
for idx, row in duplications.iterrows():
    for i in range(int(row["length"])):
        row_dict = dict(row)
        row_dict["b_num"] = idx
        row_dict["num"] = i
        data.append(row_dict)
long_duplications = pd.DataFrame(data)

In [87]:
# Viterbi code from https://github.com/phvu/misc/blob/master/viterbi/viterbi.py

class Decoder(object):
    def __init__(self, initialProb, transProb, obsProb):
        self.N = initialProb.shape[0]
        self.initialProb = initialProb
        self.transProb = transProb
        self.obsProb = obsProb
        assert self.initialProb.shape == (self.N, 1)
        assert self.transProb.shape == (self.N, self.N)
        assert self.obsProb.shape[0] == self.N
        
    def Obs(self, obs, n=None):
        o = self.obsProb[:, obs, None]
        # if n is not None:
        #     o = self.obs_probabilities[:, n, None]
        # else:
        #     o = np.array([[p.pdf(obs)] for p in self.obsProb])
        # print(repr(o))
        return o

    def Decode(self, obs):
        #self.obs_probabilities = np.array([p.pdf(obs) for p in self.obsProb])
        global trellis
        trellis = np.zeros((self.N, len(obs)), "float128")
        backpt = np.ones((self.N, len(obs)), 'int32') * -1
                
        # initialization
        trellis[:, 0] = np.squeeze(self.initialProb * self.Obs(obs[0]))
                
        for t in range(1, len(obs)):
            trellis[:, t] = (trellis[:, t-1, None].dot(self.Obs(obs[t], t).T) * self.transProb).max(0)
            backpt[:, t] = (np.tile(trellis[:, t-1, None], [1, self.N]) * self.transProb).argmax(0)
        # termination
        tokens = [trellis[:, -1].argmax()]
        for i in range(len(obs)-1, 0, -1):
            tokens.append(backpt[tokens[-1], i])
        return tokens[::-1]
    
decoder = Decoder(start_probability, trans_mat, obs_mat)

In [88]:
long_segmented_duplications = long_duplications.copy()

#results = {}
#with ProcessPoolExecutor() as executor:
#    # Add each segmentation task to the process pool
#    for n in duplications:
#        if n in ["pos", "length"]:
#            continue
#        results[n] = executor.submit(decoder.Decode, long_duplications[n].astype("int"))
#    
#    print("Started all tasks...")
#    # Get results
#    for n in duplications:
#        if n in ["pos", "length"]:
#            continue
#        long_segmented_duplications[n] = results[n].result()
        
#
# To run serially without the ProcessPoolExecutor, comment the code above and uncomment the code below:
#

for n in duplications:
    if n in ["pos", "length"]:
        continue
    long_segmented_duplications[n] = decoder.Decode(long_duplications[n].astype("int"))
    
segmented_duplications = long_segmented_duplications.groupby("b_num").mean()[list(coverage.columns)].astype("int")
segmented_duplications = segmented_duplications.reindex(list(duplications.index)) # Restore original ordering

In [96]:
merged_duplications = coverage.copy()
for col in list(coverage):
    merged_duplications[col] = duplications[col] | segmented_duplications[col]

In [98]:
merged_duplications.to_csv("../Data/Mutation_data/Overlaid_segmented_and_raw_duplications.tsv", sep="\t")

In [100]:
def parse_duplications(ser):
    ser = pd.concat([ser, ser])
    flag = 0
    start = None
    i = 0
    while True:
        if ser[i] == 0:
            break
        else:
            i += 1
    for idx, s in ser[i:].items():
        if s == 1:
            if flag == 0:
                flag = 1
                start = idx
        elif s == 0:
            if flag == 1:
                flag = 0
                yield start, last_idx
        last_idx = idx

In [102]:
from collections import OrderedDict
data = OrderedDict()
for c in coverage:
    for start, end in parse_duplications(merged_duplications[c]):
        dat = (c, start, end)
        if dat not in data:
            data[dat] = 1
            
duplication_calls = pd.DataFrame(list(data.keys()), columns=["Strain", "first_gene", "last_gene"])
duplication_calls.to_csv("../Data/Mutation_data/duplication_calls.tsv", sep="\t", index=None)